In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

In [ ]:
!pip install tensorflow


# **Using LSTM**

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
x = pd.read_csv('Train_data.csv')
y = pd.read_csv('Test_data.csv')

X_train = train_df['clean_text']
y_train = train_df['IsHatespeech']
X_test = test_df['clean_text']
y_test = test_df['IsHatespeech']

tfidf_vectorizer = TfidfVectorizer(max_features=2500)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Removed the erroneous train_test_split
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Use the already vectorized data
X_train = X_train_tfidf
X_test = X_test_tfidf

X_train = sparse.csr_matrix(X_train.sorted_indices())
X_test = sparse.csr_matrix(X_test.sorted_indices())

# Convert sparse matrices to dense arrays for LSTM
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Reshape data for LSTM (assuming each text is a single "time step")
X_train_reshaped = X_train_dense.reshape(X_train_dense.shape[0], 1, X_train_dense.shape[1])
X_test_reshaped = X_test_dense.reshape(X_test_dense.shape[0], 1, X_test_dense.shape[1])

# Define the LSTM model architecture
model = Sequential()
model.add(LSTM(64, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))  # Adjust input shape
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

Epoch 1/10
233/233 [==============================] - 7s 16ms/step - loss: 0.6444 - accuracy: 0.7049 - val_loss: 0.5680 - val_accuracy: 0.7476
Epoch 2/10
233/233 [==============================] - 4s 19ms/step - loss: 0.4812 - accuracy: 0.7922 - val_loss: 0.4955 - val_accuracy: 0.7573
Epoch 3/10
233/233 [==============================] - 3s 14ms/step - loss: 0.4060 - accuracy: 0.8187 - val_loss: 0.4908 - val_accuracy: 0.7648
Epoch 4/10
233/233 [==============================] - 3s 13ms/step - loss: 0.3680 - accuracy: 0.8414 - val_loss: 0.4996 - val_accuracy: 0.7615
Epoch 5/10
233/233 [==============================] - 3s 13ms/step - loss: 0.3406 - accuracy: 0.8480 - val_loss: 0.5170 - val_accuracy: 0.7573
Epoch 6/10
233/233 [==============================] - 4s 17ms/step - loss: 0.3222 - accuracy: 0.8603 - val_loss: 0.5402 - val_accuracy: 0.7530
Epoch 7/10
233/233 [==============================] - 3s 13ms/step - loss: 0.3091 - accuracy: 0.8657 - val_loss: 0.5577 - val_accuracy: 0.7492

# **Improvisation 1**
* Increased Epoch and Batch Size values

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train, epochs=15, batch_size=64, validation_data=(X_test_reshaped, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

Epoch 1/15
117/117 [==============================] - 37s 261ms/step - loss: 0.6496 - accuracy: 0.6110 - val_loss: 0.5278 - val_accuracy: 0.7594
Epoch 2/15
117/117 [==============================] - 28s 243ms/step - loss: 0.4162 - accuracy: 0.8190 - val_loss: 0.4519 - val_accuracy: 0.7884
Epoch 3/15
117/117 [==============================] - 34s 290ms/step - loss: 0.2751 - accuracy: 0.8955 - val_loss: 0.4789 - val_accuracy: 0.7777
Epoch 4/15
117/117 [==============================] - 29s 247ms/step - loss: 0.1855 - accuracy: 0.9362 - val_loss: 0.5517 - val_accuracy: 0.7691
Epoch 5/15
117/117 [==============================] - 29s 252ms/step - loss: 0.1211 - accuracy: 0.9600 - val_loss: 0.6394 - val_accuracy: 0.7701
Epoch 6/15
117/117 [==============================] - 31s 268ms/step - loss: 0.0821 - accuracy: 0.9754 - val_loss: 0.7556 - val_accuracy: 0.7621
Epoch 7/15
117/117 [==============================] - 28s 242ms/step - loss: 0.0578 - accuracy: 0.9825 - val_loss: 0.8749 - val_ac

# **Improvisation 2**
* Used a bidirectional LSTM (Long Short-Term Memory) model for Improvisation 2.
* The main reason for choosing this architecture is its ability to capture dependencies in both forward and backward directions, which is crucial for understanding the context in sequential data. This bidirectional approach enhances the model's performance in tasks requiring a deep understanding of the temporal context, leading to more accurate and meaningful results.


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

X_train = train_df['clean_text']
y_train = train_df['IsHatespeech']
X_test = test_df['clean_text']
y_test = test_df['IsHatespeech']

tfidf_vectorizer = TfidfVectorizer(max_features=2500)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


X_train = sparse.csr_matrix(X_train.sorted_indices())
X_test = sparse.csr_matrix(X_test.sorted_indices())

# Convert sparse matrices to dense arrays for LSTM
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Reshape data for LSTM (assuming each text is a single "time step")
X_train_reshaped = X_train_dense.reshape(X_train_dense.shape[0], 1, X_train_dense.shape[1])
X_test_reshaped = X_test_dense.reshape(X_test_dense.shape[0], 1, X_test_dense.shape[1])

# Define the LSTM model architecture
model = Sequential()
model.add(Bidirectional(LSTM(128, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train, epochs=15, batch_size=64, validation_data=(X_test_reshaped, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

Epoch 1/15
117/117 [==============================] - 88s 648ms/step - loss: 0.6106 - accuracy: 0.6490 - val_loss: 0.4767 - val_accuracy: 0.7771
Epoch 2/15
117/117 [==============================] - 59s 508ms/step - loss: 0.3673 - accuracy: 0.8436 - val_loss: 0.4547 - val_accuracy: 0.7798
Epoch 3/15
117/117 [==============================] - 57s 485ms/step - loss: 0.2269 - accuracy: 0.9133 - val_loss: 0.5168 - val_accuracy: 0.7760
Epoch 4/15
117/117 [==============================] - 59s 502ms/step - loss: 0.1403 - accuracy: 0.9518 - val_loss: 0.6182 - val_accuracy: 0.7642
Epoch 5/15
117/117 [==============================] - 56s 480ms/step - loss: 0.0885 - accuracy: 0.9726 - val_loss: 0.7490 - val_accuracy: 0.7610
Epoch 6/15
117/117 [==============================] - 58s 495ms/step - loss: 0.0605 - accuracy: 0.9828 - val_loss: 0.8705 - val_accuracy: 0.7513
Epoch 7/15
117/117 [==============================] - 53s 454ms/step - loss: 0.0402 - accuracy: 0.9889 - val_loss: 1.0169 - val_ac

# **Improvisation 3**
* Implemented increased dropout and L2 regularization in our model.
* The reason for this is to enhance the model's ability to generalize by preventing overfitting. Increased dropout helps in randomly dropping units during training, which forces the network to learn more robust features that are not reliant on any specific neurons. L2 regularization, on the other hand, penalizes large weights, encouraging the model to keep the weights small and reducing the risk of overfitting. Together, these techniques contribute to a more reliable and generalized model performance.

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping



# Define the LSTM model architecture
model = Sequential()
model.add(Bidirectional(LSTM(128, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True)))
model.add(Dropout(0.7))  # Increased dropout
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.7))  # Increased dropout
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01)))  # Added L2 regularization

# Compile the model with RMSprop optimizer and a lower learning rate
optimizer = RMSprop(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
model.fit(X_train_reshaped, y_train, epochs=15, batch_size=64, validation_data=(X_test_reshaped, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

Epoch 1/15
117/117 [==============================] - 80s 607ms/step - loss: 0.7110 - accuracy: 0.5089 - val_loss: 0.7095 - val_accuracy: 0.5016
Epoch 2/15
117/117 [==============================] - 51s 432ms/step - loss: 0.7080 - accuracy: 0.5118 - val_loss: 0.7068 - val_accuracy: 0.5016
Epoch 3/15
117/117 [==============================] - 51s 430ms/step - loss: 0.7054 - accuracy: 0.5122 - val_loss: 0.7044 - val_accuracy: 0.5016
Epoch 4/15
117/117 [==============================] - 53s 452ms/step - loss: 0.7032 - accuracy: 0.5122 - val_loss: 0.7024 - val_accuracy: 0.5016
Epoch 5/15
117/117 [==============================] - 50s 429ms/step - loss: 0.7013 - accuracy: 0.5122 - val_loss: 0.7006 - val_accuracy: 0.5016
Epoch 6/15
117/117 [==============================] - 51s 441ms/step - loss: 0.6996 - accuracy: 0.5122 - val_loss: 0.6990 - val_accuracy: 0.5016
Epoch 7/15
117/117 [==============================] - 51s 433ms/step - loss: 0.6982 - accuracy: 0.5122 - val_loss: 0.6977 - val_ac

In [ ]:
!pip install transformers

# **bert-base-uncased**
This model offers a balanced trade-off between size and performance, making it suitable for tasks where computational resources are a consideration but high accuracy and contextual understanding are still required. By leveraging 'bert-base-uncased,' we can enhance our text processing and analysis capabilities, leading to more accurate and meaningful outcomes.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Load your data
train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

X_train = train_df['clean_text']
y_train = train_df['IsHatespeech']
X_test = test_df['clean_text']
y_test = test_df['IsHatespeech']

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

# Tokenize and encode the text data
def encode_text(texts, tokenizer, max_length=128):
    encodings = tokenizer(
        texts.tolist(),
        truncation=True,           #decimal number is typecast as an integer
        padding='max_length',      #The max_length argument controls the length of the padding and truncation.
        max_length=max_length,
        return_tensors='tf'        #it indicates that you want the output in the form of TensorFlow tensors.
    )
    return encodings

X_train_encodings = encode_text(X_train, tokenizer)
X_test_encodings = encode_text(X_test, tokenizer)

# Convert labels to TensorFlow tensors
y_train_tensor = tf.constant(y_train.tolist())
y_test_tensor = tf.constant(y_test.tolist())

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(
    X_train_encodings,
    y_train_tensor,
    epochs=3,  # Adjust as needed
    batch_size=16,  # Adjust as needed
    validation_data=(X_test_encodings, y_test_tensor)
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_encodings, y_test_tensor)
print("Loss:", loss)
print("Accuracy:", accuracy)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
466/466 [==============================] - 278s 466ms/step - loss: 0.4908 - accuracy: 0.7647 - val_loss: 0.4522 - val_accuracy: 0.7868
Epoch 2/3
466/466 [==============================] - 209s 449ms/step - loss: 0.3518 - accuracy: 0.8505 - val_loss: 0.4501 - val_accuracy: 0.7916
Epoch 3/3
59/59 [==============================] - 16s 273ms/step - loss: 0.5604 - accuracy: 0.7889
Loss: 0.5604289770126343
Accuracy: 0.7889366149902344


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoTokenizer
import tensorflow as tf
from sklearn.metrics import f1_score

# Load your data
train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

X_train = train_df['clean_text']
y_train = train_df['IsHatespeech']
X_test = test_df['clean_text']
y_test = test_df['IsHatespeech']

# Use a more powerful BERT variant
model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

# Tokenize and encode the text data with a longer sequence length
def encode_text(texts, tokenizer, max_length=256):  # Increase max_length
    encodings = tokenizer(
        texts.tolist(),
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='tf'
    )
    return encodings

X_train_encodings = encode_text(X_train, tokenizer)
X_test_encodings = encode_text(X_test, tokenizer)

# Convert labels to TensorFlow tensors
y_train_tensor = tf.constant(y_train.tolist())
y_test_tensor = tf.constant(y_test.tolist())

# Compile the model with a lower learning rate and a different optimizer
optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-5)  # Use AdamW
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model for more epochs and with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,  # Stop if no improvement for 3 epochs
    restore_best_weights=True
)

model.fit(
    {key: val for key, val in X_train_encodings.items()},
    y_train_tensor,
    epochs=10,  # Increase epochs
    batch_size=16,
    validation_data=({key: val for key, val in X_test_encodings.items()}, y_test_tensor),
    callbacks=[early_stopping]
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_encodings, y_test_tensor)
print("Loss:", loss)
print("Accuracy:", accuracy)

# Predict on the test set
y_pred_logits = model.predict(X_test_encodings).logits
y_pred = tf.argmax(y_pred_logits, axis=1).numpy()

# Calculate F1 score
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
466/466 [==============================] - 503s 934ms/step - loss: 0.5072 - accuracy: 0.7535 - val_loss: 0.4627 - val_accuracy: 0.7873
Epoch 2/10
466/466 [==============================] - 429s 920ms/step - loss: 0.3913 - accuracy: 0.8290 - val_loss: 0.4418 - val_accuracy: 0.7846
Epoch 3/10
466/466 [==============================] - 422s 905ms/step - loss: 0.2832 - accuracy: 0.8811 - val_loss: 0.4732 - val_accuracy: 0.7922
Epoch 4/10
466/466 [==============================] - 428s 919ms/step - loss: 0.1746 - accuracy: 0.9346 - val_loss: 0.6030 - val_accuracy: 0.7701
Epoch 5/10
59/59 [==============================] - 34s 573ms/step - loss: 0.4418 - accuracy: 0.7846
Loss: 0.4417506754398346
Accuracy: 0.7846401929855347
59/59 [==============================] - 43s 573ms/step
F1 Score: 0.7916883116883117
